<a href="https://colab.research.google.com/github/juanalvarez96/KAGGLE_DATASCIENCE/blob/master/Kaggle_winners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Libraries
%matplotlib inline
#PyTorch, of course
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
#We will need torchvision transforms for data augmentation

import torchvision.models as models
### utilities
# tool to print a nice summary of a network, similary to keras' summary
from torchsummary import summary

# library to do bash-like wildcard expansion
import glob
import matplotlib.pyplot as plt

# others
import numpy as np
import random
from PIL import Image
from IPython.display import display
from tqdm import tqdm_notebook
from torchvision import datasets
from google.colab import drive
drive.mount('/content/drive')

train_path = "/content/drive/My Drive/polytech-ds-2019/polytech-ds-2019/training"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Import Data and generate DataLoaders**
This process will be carried out using tbe torchvisison datasets set of function for importing data.\
Since the provided data is shuffled, we will organize each picture in folders. This implies, for every image from the training and validation folders, we will move it to a class folder.


1.   Create for the validation folder 10 folders following the structure: class0, class1, ..., class10
2.   Move every image from validation folder to its respective class folder. Move 01_xyz.jpg to class1, move 02_xyz.jpg to class2, ...
3. Repeat these two steps fro the training floder



In [0]:
transformations=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [0]:
# Needed to organize data for the classifier.
# Only do ONE TIME for the training
% cd /content/drive/My\ Drive/polytech-ds-2019/polytech-ds-2019/training
#! for i in `seq 0 10`; do mkdir class${i}; done
#! for i in `seq 0 10`; do mv ${i}_* class$i; done
! for i in `seq 0 10`; do ls -lh class${i}/ | wc -l; done

/content/drive/My Drive/polytech-ds-2019/polytech-ds-2019/training
995
430
1501
987
849
1326
441
281
856
1501
710


In [0]:
# Needed to organize data for the classifier.
# Only do ONE TIME for the validation
#! for i in `seq 0 10`; do mkdir ../validation/class${i}; done
#! for i in `seq 0 10`; do mv ../validation/${i}_* ../validation/class$i; done
! for i in `seq 0 10`; do ls -lh ../validation/class${i}/ | wc -l; done

365
145
502
329
327
451
148
100
348
504
233


In [0]:
# Load in each dataset and apply transformations using
# the torchvision.datasets as datasets library
% cd /content/drive/My\ Drive/polytech-ds-2019/polytech-ds-2019/training
data_set = datasets.ImageFolder(root="./", transform = transformations)
val_set = datasets.ImageFolder(root="../validation/", transform = transformations)

/content/drive/My Drive/polytech-ds-2019/polytech-ds-2019/training


In [0]:
data_set

In [0]:
data_loader = torch.utils.data.DataLoader(data_set, batch_size=32, shuffle=True)

In [0]:
tensor, labels = next(iter(test_loader))
tensor.size()

torch.Size([64, 3, 224, 224])

In [0]:
# Visualize image
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(data_loader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
class_names = data_set.classes
imshow(out, title=[class_names[x] for x in classes])


In [0]:
pip install --upgrade fastai

In [0]:
pip install --upgrade torchvision

In [0]:
!pip install torchvision==0.1.9
!pip install fastai
!pip install torchtext==0.2.3

In [0]:
from fastai.vision import *


TypeError: ignored

/bin/bash: fastai: command not found
